# A primer on Pathlib

[`Pathlib`](https://docs.python.org/3/library/pathlib.html) is one of the most amazing Python libraries. It's incredibly beautiful, well designed and written and convenient for day to day work. It's about file and directory management. It has a highly intuitive and beautiful API. Wait, "beautiful API"?

![simpsons nerd](https://media1.tenor.com/images/32bc92e5ab305b8c5ad3edac00de47e9/tenor.gif?itemid=7884166)

`Pathlib` is a builtin library **only** in Python 3, if you want to use it in Python 2 you need to install a 3rd party library, which is not official and might have some inconsistencies with the official Py3 one.

Pathlib API is inspired by the regular usage of paths in \*nix operating systems, which makes it intuitive and familiar for developers. In this notebook, we'll explore different situations where Pathlib shows all its power and strength.

### File exists? Is it a file or a directory?

How can you check if a file exists in Python? Regular solution ([most popular StackOverflow answer](https://stackoverflow.com/questions/82831/how-to-check-whether-a-file-exists)) shows something like:

In [52]:
import os
os.path.exists('alice.txt')

True

In [53]:
os.path.isfile('alice.txt')

True

In [54]:
os.path.isdir('alice.txt')

False

Pathlib solution:

In [55]:
from pathlib import Path
path = Path('alice.txt')

In [56]:
path.exists()

True

In [57]:
path.is_file()

True

In [58]:
path.is_dir()

False

Much cleaner, intuitive and Object Oriented, isn't it? With `Pathlib`, you just need to create a `Path` object, which is an "abstract interface" that bases its concrete implementation according to the Operating System:

![pathlib hierarchy](https://docs.python.org/3/_images/pathlib-inheritance.png)

### Concatenating Paths

Joining "parts" of paths can be tedious because you need to be aware of the Operating System syntax to express paths. For example, Linux and Mac use forward slash `/` to join paths, while windows uses backslashes `\`. Example of paths:

* Linux/Mac: `data/subdir/0005.txt`
* Windows: `data\subdir\0005.txt`

So, if you have those paths expressed in parts, it's hard to combine them to read the content:

In [59]:
BASE_DIR = 'data'
SUBDIR = 'subdir'
FILE_NAME = '0005.txt'

Using the `os` module:

In [60]:
os.path.join(BASE_DIR, SUBDIR)

'data/subdir'

In [61]:
os.path.join(BASE_DIR, SUBDIR, FILE_NAME)

'data/subdir/0005.txt'

Using the pathlib module:

In [62]:
BASE_PATH = Path(BASE_DIR)

In [63]:
BASE_PATH / SUBDIR / FILE_NAME

PosixPath('data/subdir/0005.txt')

In this case, the `/` operator (division for integers) is overloaded and assigned a different "meaning" per Pathlib's API. Isn't `/` intuitive to join paths? And given a full path, it's much easier to split it into each part:

In [64]:
p1 = Path('/home/rmotr/code/python/main.py')
p2 = Path('C:/home/rmotr/code/python/main.py')

In [65]:
p1.parts

('/', 'home', 'rmotr', 'code', 'python', 'main.py')

In [66]:
p2.parts

('C:', 'home', 'rmotr', 'code', 'python', 'main.py')

In [67]:
dirp1 = ('/home/rmotr/code/python/main.py')
dirp1.split(os.path.sep)

['', 'home', 'rmotr', 'code', 'python', 'main.py']

In [68]:
dirp2 = ('C:/home/rmotr/code/python/main.py')
dirp2.split(os.path.sep)

['C:', 'home', 'rmotr', 'code', 'python', 'main.py']

Plus a few other convenient methods:

In [69]:
p1.root

'/'

In [70]:
p2.root

''

In [71]:
p1.parents[0]

PosixPath('/home/rmotr/code/python')

In [72]:
p1.parents[1]

PosixPath('/home/rmotr/code')

In [73]:
p2.parents[0]

PosixPath('C:/home/rmotr/code/python')

In [74]:
p1.parent

PosixPath('/home/rmotr/code/python')

In [75]:
p2.parent

PosixPath('C:/home/rmotr/code/python')

In [76]:
os.path.dirname(dirp1)

'/home/rmotr/code/python'

In [77]:
os.path.dirname(dirp2)

'C:/home/rmotr/code/python'

In [78]:
os.path.basename(dirp1)

'main.py'

In [79]:
os.path.basename(dirp2)

'main.py'

### Extracting suffixes

How can you get the suffix and name of the file under `/home/rmotr/code/python/main.py`? Regular `os` solution:

In [80]:
os.path.splitext('/home/rmotr/code/python/main.py')

('/home/rmotr/code/python/main', '.py')

`os.path.splitext` isn't really intuitive, what about Pathlib?

In [81]:
p = Path('/home/rmotr/code/python/main.py')

In [82]:
p.suffix

'.py'

In [83]:
p.stem

'main'

### Getting current dir and Home dir

This is one of the [proposed solutions](https://stackoverflow.com/questions/5137497/find-current-directory-and-files-directory) to get the current path using the `os` module:

```python
os.path.dirname(os.path.realpath(__main__))
```
Not pretty. What about the other one:

In [84]:
os.getcwd()

'/app'

A little better. What about Pathlib?

In [85]:
Path.cwd()  # cwd: current working dir

PosixPath('/app')

What about the Home directory:

In [86]:
from os.path import expanduser
expanduser("~")

'/root'

`expanduser` 😕

Pathlib:

In [87]:
Path.home()

PosixPath('/root')

![feels good](https://user-images.githubusercontent.com/872296/37731886-3856a696-2d22-11e8-9fd4-05be4b6672df.png)


### Creating directories

Let's create a new subdir using `os`:

In [88]:
new_dir_name = 'new-dir-1'
new_dir = os.path.join(os.path.join(BASE_DIR, SUBDIR), new_dir_name)
os.makedirs(new_dir)
print(new_dir)

FileExistsError: [Errno 17] File exists: 'data/subdir/new-dir-1'

In [89]:
os.path.exists(new_dir)

True

But with Pathlib, as expected, it's a lot more intuitive:

In [ ]:
p = Path(BASE_DIR) / SUBDIR / 'new-dir-2' / 'test.txt'
print(p)
with p.open('w', encoding='utf-8') as f:
    f.write('some stuff in here')

In [93]:
p.mkdir()

FileNotFoundError: [Errno 2] No such file or directory: '/home/rmotr/code/python/main.py'

In [94]:
p.exists()

False

### Searching for things

This is the last strike for `os`. There aren't many ways of doing what we'll do without Pathlib. So, we'll just skip the `os` versions.

Looking for all the `txt` files in a directory:

In [95]:
sorted(Path('data').glob('*.txt'))

[PosixPath('data/0001.txt'),
 PosixPath('data/0002.txt'),
 PosixPath('data/0003.txt'),
 PosixPath('data/0004.txt'),
 PosixPath('data/0005.txt')]

Now, looking for files **recursively** (hold my beer):

In [ ]:
sorted(Path('data').glob('**/*.txt'))

It also found stuff under `subdir`!

![mind blown](http://www.reactiongifs.com/wp-content/uploads/2013/10/tim-and-eric-mind-blown.gif)

You've now seen how useful and clean the Pathlib API is. We encourage you to check other useful methods and practice with it: https://docs.python.org/3/library/pathlib.html